In [21]:
# Imports
import os
import time
import glob
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imutils.video import VideoStream
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import pandas as pd
from functions import getLargestContour, getContourFeatures, unpackAdvancedFeatures


In [22]:
# Constants
data_path = 'images'
mappings = {"r": "r", "p": "p", "s": "s"}
frame_size = (320, 240)
feature_names = [
    'area',
    'perimeter',
    'aspect_ratio',
    'extent',
    'convex_hull_area',           # Area of the convex hull
    'hull_perimeter',              # Perimeter of the convex hull
    'solidity',                    # Ratio of area to convex hull area
    'circularity',                 # Measure of how circular the shape is
    'eccentricity',                # Measure of the elongation of the shape
    'num_convexity_defects',       # Number of convexity defects
    'max_defect_depth',            # Maximum depth of convexity defects
    'major_axis_length',           # Length of the major axis of the fitted ellipse
    'minor_axis_length',           # Length of the minor axis of the fitted ellipse
    'orientation',                  # Orientation angle of the fitted ellipse
    'hu_moment',
    'centroid_x',
    'centroid_y'
]


categories = ['Rock', 'Paper', 'Scissor']
size = (320, 240)

lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Lower boundary of skin tone in HSV
upper_skin = np.array([20, 255, 255], dtype=np.uint8)  # Upper boundary of skin tone in HSV




In [23]:
# Initialize lists
data = np.empty((0, len(feature_names)), float)  # To store feature vectors
target = []  # To store labels
images = []  # List to store images
labels = []  # List to store corresponding labels

In [24]:
# Key mappings
kMappings = {ord(key): value for key, value in mappings.items()}

In [25]:
for category in categories:
    # Get the corresponding label from the mappings dictionary
    label = mappings[category[0].lower()]  # 'r' -> 0, 'p' -> 1, 's' -> 2
    
    # Form the path to the current category's images
    folder_path = os.path.join(data_path, category)
    
    # Get all image file paths in the current folder
    image_paths = glob.glob(os.path.join(folder_path, '*.jpg'))
    
    print(f"[INFO] Processing {category} images, found: {len(image_paths)}")
    
    # Loop over each image path
    for image_path in image_paths:
        # Load the image using OpenCV
        img = cv.imread(image_path)
        
        if img is None:
            print(f"[WARNING] Unable to load image {image_path}. Skipping.")
            continue
        
        # Resize the image to a standard frame size
        img = cv.resize(img, frame_size)
        
        # Append the image and its label
        images.append(img)  # Store the image
        labels.append(label)  # Store the corresponding label

# Convert images and labels to NumPy arrays for consistency
images = np.array(images)
labels = np.array(labels)

# Display some information about the dataset
print(f"[INFO] Number of images loaded: {len(images)}")
print(f"[INFO] Shape of images array: {images.shape}")
print(f"[INFO] Number of labels: {len(labels)}")

[INFO] Processing Rock images, found: 54
[INFO] Processing Paper images, found: 40
[INFO] Processing Scissor images, found: 39
[INFO] Number of images loaded: 133
[INFO] Shape of images array: (133, 240, 320, 3)
[INFO] Number of labels: 133


In [26]:
# Loop over each image and process for background removal
for i, img in enumerate(images):
    # Convert the image to the HSV color space
    hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    # Create a binary mask where skin color is within the specified range
    masked_img = cv.inRange(hsv_img, lower_skin, upper_skin)

    # Apply morphological operations to clean up the mask (remove small noises)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5,5))
    masked_img = cv.morphologyEx(masked_img, cv.MORPH_CLOSE, kernel)
    masked_img = cv.erode(masked_img, None, iterations=2)
    masked_img = cv.dilate(masked_img, None, iterations=2)

    # Create the result image by applying the mask to the original image
    masked_color_img = cv.bitwise_and(img, img, mask=masked_img)

    # Convert the result to grayscale to proceed with feature extraction
    gray_img = cv.cvtColor(masked_color_img, cv.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image to reduce noise
    blurred_img = cv.GaussianBlur(gray_img, (5,5), 0)

    # Apply binary thresholding to convert the image to binary (black & white)
    _, bw_img = cv.threshold(blurred_img, 10, 255, cv.THRESH_BINARY_INV)

    # Apply morphological operations to remove small noise and close gaps
    bw_img = cv.morphologyEx(bw_img, cv.MORPH_CLOSE, kernel)

    # Apply high-pass filter to remove low-frequency background
    bw_img = cv.morphologyEx(bw_img, cv.MORPH_GRADIENT, kernel)

    # Check if any foreground remains after thresholding
    if cv.countNonZero(bw_img) == 0:
        print(f"[WARNING] No foreground detected in image {i}. Skipping.")
        continue

    # Find the largest contour from the binary image
    try:
        contour = getLargestContour(bw_img)
    except ValueError:
        print(f"[WARNING] Could not find contours in image {i}. Skipping.")
        continue

    # Extract features from the largest contour
    features, defects, feature_vector = getContourFeatures(contour)
    
    # Append the features to the data matrix
    data = np.append(data, np.array([features]), axis=0)
    
    # Append the corresponding label from the 'labels' array
    target.append(labels[i])

    # draw the outline of the object
    cv.drawContours(img, [contour], -1, (0, 255, 0), 1)
    cv.drawContours(masked_img, [contour], -1, (0, 255, 0), 1)
    cv.drawContours(masked_color_img, [contour], -1, (0, 255, 0), 1)
    cv.drawContours(bw_img, [contour], -1, (0, 255, 0), 1)

    # point out hull defects
    if defects is not None:
        for s,e,f,d in defects:
            start = tuple(contour[s])
            end = tuple(contour[e])
            far = tuple(contour[f])
            cv.line(img,start,end,[0,255,255],2)
            cv.circle(img,far,5,[0,0,255],-1)

    # Optionally, display the original image, mask, and result for visualization
    # cv.imshow("Original Image", img)
    # cv.imshow("HSV Image", hsv_img)
    # cv.imshow("Skin Mask", masked_img)
    # cv.imshow("Result after Background Removal", masked_color_img)
    # cv.imshow("Grayscale Image", gray_img)
    # cv.imshow("Binary Image for Contour", bw_img)

    print("[INFO] contour features: {}".format(feature_vector))

    # Wait for a key press and quit if 'q' is pressed
    k = cv.waitKey(1) & 0xFF
    if k == ord("q"):
        break

# Convert the target list to a NumPy array
target = np.array(target)

# Display the shape of the resulting feature dataset
print(f"[INFO] Feature extraction complete.")
print(f"[INFO] Shape of data matrix: {data.shape}")
print(f"[INFO] Number of target labels: {target.shape[0]}")

# Create a Bunch object containing the dataset
dataset = Bunch(data=data,
                target=target,
                feature_names=feature_names,
                unique_targets=np.unique(target))

print(f"[INFO] Dataset created with shape: {dataset.data.shape}")

[INFO] contour features: [ 2.35000000e+02  1.07070312e+01  8.30468750e+00  6.15625000e+00
  3.50390625e+00  2.53125000e+00  1.55400000e+03  1.12129350e+03
  2.07826087e+00  5.65399309e-02  1.87595000e+04  5.91084302e+02
  8.28380287e-02  1.55318172e-02  9.19525560e-01  1.80000000e+01
  2.35000000e+02  2.41704208e+02  9.49970551e+01  9.96195145e+01
 -6.18644759e-01  1.92000000e+02  1.52000000e+02]
[INFO] contour features: [ 2.40000000e+02  9.73046875e+00  6.86328125e+00  5.19140625e+00
  4.03125000e+00  2.77343750e+00  1.57800000e+03  1.12721024e+03
  2.51546392e+00  6.66723002e-02  1.89490000e+04  5.92648888e+02
  8.32761623e-02  1.56065538e-02  9.27152246e-01  1.80000000e+01
  2.40000000e+02  2.42837219e+02  9.09874191e+01  9.19800797e+01
 -6.20155359e-01  1.89000000e+02  1.38000000e+02]
[INFO] contour features: [ 2.43000000e+02  1.13632812e+01  6.39843750e+00  4.33984375e+00
  4.23828125e+00  3.21875000e+00  1.65550000e+03  1.15608030e+03
  2.39805825e+00  6.50721277e-02  2.07185000e

In [27]:
unpackAdvancedFeatures(dataset.data, dataset.target)

[INFO] Number of feature vectors: 133
[INFO] Number of labels: 133
Image #   Area           Perimeter      Aspect Ratio   Extent         Convex Hull Area    Hull Perimeter      Solidity       Circularity    Eccentricity   Convexity Defects   Max Defect Depth    Major Axis Length   Minor Axis Length   Orientation    Hu moments     Centroid_x     Centroid_y     Label
0         1554.00        1121.29        2.08           0.06           18759.50            591.08              0.08           0.02           0.92           18.0                235.00              241.70              95.00               99.62          -0.62          192.00         152.00         r
1         1578.00        1127.21        2.52           0.07           18949.00            592.65              0.08           0.02           0.93           18.0                240.00              242.84              90.99               91.98          -0.62          189.00         138.00         r
2         1655.50        1156.08      

In [28]:
# Export to csv
"""Export the feature data and target labels to a CSV file."""
# Create a DataFrame from the feature data
f = pd.DataFrame(data, columns=feature_names)  # Use the feature_names list
# Ad the target labels as a new column
f['Label'] = target
# Expot to CSV
filename = 'data/hand_gesture_features.csv'
f.to_csv(filename, index=False)
print(f"[INFO] Data exported to {filename}")

[INFO] Data exported to data/hand_gesture_features.csv
